In [1]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# 1. Cargar el conjunto de datos
# Usamos el conjunto de datos Iris como ejemplo
iris = datasets.load_iris()
X = iris.data  # Características
y = iris.target  # Etiquetas

# Crear un DataFrame para mejor visualización
feature_names = [name.replace(" (cm)", "").replace(" ", "_") for name in iris.feature_names]
df = pd.DataFrame(X, columns=feature_names)
df['Target'] = y

# Mostrar las primeras filas del conjunto de datos
print("Primeras filas del conjunto de datos:")
print(df.head())

# 2. Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 3. Crear una función para probar diferentes modelos
def evaluate_models(models, X_train, X_test, y_train, y_test):
    """
    Entrenar y evaluar diferentes modelos de clasificación.

    Args:
        models (dict): Diccionario con los nombres y las instancias de los modelos.
        X_train, X_test, y_train, y_test: Conjuntos de datos de entrenamiento y prueba.

    Returns:
        None. Imprime el rendimiento de cada modelo.
    """
    for name, model in models.items():
        print(f"\nEntrenando y evaluando el modelo: {name}")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),  # Escalado de datos
            ('classifier', model)         # Modelo
        ])

        # Entrenar el modelo
        pipeline.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = pipeline.predict(X_test)

        # Evaluar el modelo
        print("Matriz de confusión:")
        print(confusion_matrix(y_test, y_pred))
        print("Reporte de clasificación:")
        print(classification_report(y_test, y_pred))

# 4. Definir los modelos a probar
models = {
    'Logistic Regression': LogisticRegression(max_iter=200, random_state=42),
    'Support Vector Machine': SVC(kernel='rbf', probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# 5. Evaluar los modelos
evaluate_models(models, X_train, X_test, y_train, y_test)

# 6. Optimizar hiperparámetros de un modelo (ejemplo con Random Forest)
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10]
}

# Crear un pipeline con escalado y modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Usar GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("\nMejores hiperparámetros para Random Forest:")
print(grid_search.best_params_)

# Evaluar el mejor modelo en el conjunto de prueba
y_pred_optimized = grid_search.best_estimator_.predict(X_test)
print("\nReporte de clasificación del modelo optimizado:")
print(classification_report(y_test, y_pred_optimized))


Primeras filas del conjunto de datos:
   sepal_length  sepal_width  petal_length  petal_width  Target
0           5.1          3.5           1.4          0.2       0
1           4.9          3.0           1.4          0.2       0
2           4.7          3.2           1.3          0.2       0
3           4.6          3.1           1.5          0.2       0
4           5.0          3.6           1.4          0.2       0

Entrenando y evaluando el modelo: Logistic Regression
Matriz de confusión:
[[15  0  0]
 [ 0 14  1]
 [ 0  3 12]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.82      0.93      0.88        15
           2       0.92      0.80      0.86        15

    accuracy                           0.91        45
   macro avg       0.92      0.91      0.91        45
weighted avg       0.92      0.91      0.91        45


Entrenando y evaluando el modelo: Support Vector Machine
